In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        #invert
        image=cv2.flip(image,1)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
def extract_keypoints(results):
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])
extract_keypoints(results)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [8]:
#path folder
data_path=os.path.join('new_5')

#actions
actions=np.array(['palm','peace','rock','thumbs up','thumbs down'])

#100 datasets
no_sequence=100
sequence_length=5

In [9]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(data_path,action,str(sequence)))
        except:
            pass
        

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    for action in actions:
        for sequence in range(no_sequence):
            for frame_num in range(sequence_length):
                

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(200)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen 
                    cv2.imshow('OpenCV Feed', image)
                
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(data_path, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                #invert
                image=cv2.flip(image,1)

                # Show to screen
                cv2.imshow('OpenCV Feed', image)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map

{'palm': 0, 'peace': 1, 'rock': 2, 'thumbs up': 3, 'thumbs down': 4}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(data_path, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(data_path, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(500, 5, 126)

In [16]:
np.array(labels).shape

(500,)

In [17]:
X = np.array(sequences)
X.shape

(500, 5, 126)

In [18]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
X_test.shape

(25, 5, 126)

In [21]:
y_test.shape

(25, 5)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
from tensorflow.keras.models import load_model
model=load_model('mp_hand_gesture')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 42)                0         
                                                                 
 dense_11 (Dense)            (None, 64)                2752      
                                                                 
 dense_12 (Dense)            (None, 128)               8320      
                                                                 
 dense_13 (Dense)            (None, 512)               66048     
                                                                 
 dense_14 (Dense)            (None, 64)                32832     
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 10)               

In [24]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(5,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
15/15 [==============================] - 2s 8ms/step - loss: 1.6089 - categorical_accuracy: 0.1832
Epoch 2/2000
15/15 [==============================] - 0s 8ms/step - loss: 1.5876 - categorical_accuracy: 0.4274
Epoch 3/2000
15/15 [==============================] - 0s 7ms/step - loss: 1.2601 - categorical_accuracy: 0.5389
Epoch 4/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.5620 - categorical_accuracy: 0.7621
Epoch 5/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.5028 - categorical_accuracy: 0.8147
Epoch 6/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.2681 - categorical_accuracy: 0.9474
Epoch 7/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.2888 - categorical_accuracy: 0.9053
Epoch 8/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.1439 - categorical_accuracy: 0.9705
Epoch 9/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0590 - categorical_accuracy:

15/15 [==============================] - 0s 8ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 146/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0110 - categorical_accuracy: 0.9979
Epoch 147/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0109 - categorical_accuracy: 0.9979
Epoch 148/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0112 - categorical_accuracy: 0.9979
Epoch 149/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0111 - categorical_accuracy: 0.9979
Epoch 150/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0128 - categorical_accuracy: 0.9979
Epoch 151/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 152/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0116 - categorical_accuracy: 0.9979
Epoch 153/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0103 - categorical_accura

Epoch 217/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0114 - categorical_accuracy: 0.9979
Epoch 218/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0106 - categorical_accuracy: 0.9979
Epoch 219/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0113 - categorical_accuracy: 0.9979
Epoch 220/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0117 - categorical_accuracy: 0.9979
Epoch 221/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0124 - categorical_accuracy: 0.9979
Epoch 222/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0110 - categorical_accuracy: 0.9979
Epoch 223/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 224/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0110 - categorical_accuracy: 0.9979
Epoch 225/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0102 - cat

Epoch 360/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0101 - categorical_accuracy: 0.9979
Epoch 361/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0115 - categorical_accuracy: 0.9979
Epoch 362/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0106 - categorical_accuracy: 0.9979
Epoch 363/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 364/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0107 - categorical_accuracy: 0.9979
Epoch 365/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0109 - categorical_accuracy: 0.9979
Epoch 366/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 367/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0110 - categorical_accuracy: 0.9979
Epoch 368/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0119 - cat

Epoch 503/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 504/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0111 - categorical_accuracy: 0.9979
Epoch 505/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 506/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 507/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0101 - categorical_accuracy: 0.9979
Epoch 508/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0114 - categorical_accuracy: 0.9979
Epoch 509/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0109 - categorical_accuracy: 0.9979
Epoch 510/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0103 - categorical_accuracy: 0.9979
Epoch 511/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0103 - cat

Epoch 646/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0106 - categorical_accuracy: 0.9979
Epoch 647/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0103 - categorical_accuracy: 0.9979
Epoch 648/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 649/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0109 - categorical_accuracy: 0.9979
Epoch 650/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0106 - categorical_accuracy: 0.9979
Epoch 651/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0111 - categorical_accuracy: 0.9979
Epoch 652/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0113 - categorical_accuracy: 0.9979
Epoch 653/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 654/2000
15/15 [==============================] - 0s 10ms/step - loss: 0.0104 - ca

15/15 [==============================] - 0s 8ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 789/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0103 - categorical_accuracy: 0.9979
Epoch 790/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 791/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 792/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 793/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0103 - categorical_accuracy: 0.9979
Epoch 794/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0103 - categorical_accuracy: 0.9979
Epoch 795/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0108 - categorical_accuracy: 0.9979
Epoch 796/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0101 - categorical_accura

Epoch 931/2000
15/15 [==============================] - 0s 9ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 932/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0101 - categorical_accuracy: 0.9979
Epoch 933/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 934/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 935/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 936/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0104 - categorical_accuracy: 0.9979
Epoch 937/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0109 - categorical_accuracy: 0.9979
Epoch 938/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0102 - categorical_accuracy: 0.9979
Epoch 939/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0102 - cat

15/15 [==============================] - 0s 8ms/step - loss: 0.0110 - categorical_accuracy: 0.9979
Epoch 1074/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 1075/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0106 - categorical_accuracy: 0.9979
Epoch 1076/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0108 - categorical_accuracy: 0.9979
Epoch 1077/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 1078/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0105 - categorical_accuracy: 0.9979
Epoch 1079/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0112 - categorical_accuracy: 0.9979
Epoch 1080/2000
15/15 [==============================] - 0s 8ms/step - loss: 0.0100 - categorical_accuracy: 0.9979
Epoch 1081/2000
15/15 [==============================] - 0s 7ms/step - loss: 0.0102 - categorica

KeyboardInterrupt: 

In [27]:
model.save('new_5.h5')

In [29]:
from tensorflow.keras.models import load_model
model=load_model('new_5.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             48896     
                                                                 
 lstm_1 (LSTM)               (None, 5, 128)            98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 203,525
Trainable params: 203,525
Non-trai

In [33]:
model=load_model('model5.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             48896     
                                                                 
 lstm_1 (LSTM)               (None, 5, 128)            98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 203,525
Trainable params: 203,525
Non-trai

In [1]:
#start

import cv2 #open cv
import numpy as np
import os
#from matplotlib import pyplot as plt
#import time
import mediapipe as mp

import webbrowser
import pyautogui


In [2]:
mp_holistic = mp.solutions.holistic # Holistic
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image=cv2.flip(image,1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)# COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])
extract_keypoints(results)

NameError: name 'results' is not defined

In [7]:
from tensorflow.keras.models import load_model
model=load_model('new_5.h5')
#model.summary()

#actions
actions=np.array(['palm','peace','rock','thumbs up','thumbs down'])

#100 datasets
no_sequence=100
sequence_length=5

In [8]:
import time
    

In [19]:
#main

def reload():
    #cv2.waitKey(1000)
    cv2.destroyAllWindows()
    #cap.release()
    time.sleep(2)
    startnew()

sequence = []
sentence = []
predictions = []
threshold = 0.5

classNames=['palm','peace','rock','thumbs up','thumbs down']

cap = cv2.VideoCapture(0)
# Set mediapipe model
def startnew():
    global sequence,sentence,predictions,threshold
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-5:]

            className=''

            if len(sequence) == 5:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                classID = np.argmax(res)
                className = classNames[classID]


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                #image = prob_viz(res, actions, image, colors)

            #print(className)

            # show the prediction on the frame
            #if className!='thumbs up':
            cv2.putText(image, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                     1, (200,20,198), 2, cv2.LINE_AA)


            # Show to screen

            cv2.imshow('gesture recognition', image)

            if(className=='peace'):
                webbrowser.open("https://www.youtube.com/channel/UCTAptegX07e-meUIal9MlvQ")
                className=''
                reload()
            #elif(className=='palm'):
                #pyautogui.hotkey('alt', 'tab')
                #pyautogui.hotkey('ctrl', 'tab')
                reload()
            #elif(className=='rock'):
                #webbrowser.open('google.com')
                #reload()
            elif(className=='thumbs up'):
                pyautogui.hotkey('ctrl', 'shift','esc')
                reload()
            elif(className=='thumbs down'):
                pyautogui.hotkey('alt', 'tab')
                pyautogui.hotkey('alt', 'f4')
                reload()
                        


            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


startnew()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.sol

In [18]:
def reload():
    cv2.waitKey(1000)
    cv2.destroyAllWindows()
    startnew()

sequence = []
sentence = []
predictions = []
threshold = 0.5

classNames=['palm','peace','rock','thumbs up','thumbs down']

cap = cv2.VideoCapture(0)
# Set mediapipe model
def startnew():
    global sequence,sentence,predictions,threshold
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-5:]

            className=''

            if len(sequence) == 5:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                classID = np.argmax(res)
                className = classNames[classID]


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                #image = prob_viz(res, actions, image, colors)

            #print(className)

            # show the prediction on the frame
            #if className!='thumbs up':
            cv2.putText(image, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                     1, (200,20,198), 2, cv2.LINE_AA)


            # Show to screen

            cv2.imshow('gesture recognition', image)
            if(className=='peace'):
                webbrowser.open("youtube.com")
                break
            #elif(className=='palm'):
                #pyautogui.hotkey('alt', 'tab')
                #pyautogui.hotkey('ctrl', 'tab')
                #break
            #elif(className=='rock'):
                #webbrowser.open('google.com')
                #break
            elif(className=='thumbs up'):
                pyautogui.hotkey('ctrl', 'shift','esc')
                break
            elif(className=='thumbs down'):
                pyautogui.hotkey('alt', 'tab')
                pyautogui.hotkey('alt', 'f4')
                break
        
                        



            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


startnew()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.solution_base.SolutionOutputs'>
rock
<class 'mediapipe.python.sol

In [15]:
def reload():
    cv2.waitKey(4000)
    cv2.destroyAllWindows()
    startnew()

sequence = []
sentence = []
predictions = []
threshold = 0.5

classNames=['palm','peace','rock','thumbs up','thumbs down']

cap = cv2.VideoCapture(0)
# Set mediapipe model
def startnew():
    global sequence,sentence,predictions,threshold
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-5:]

            className=''

            if len(sequence) == 5:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                classID = np.argmax(res)
                className = classNames[classID]


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                #image = prob_viz(res, actions, image, colors)

            #print(className)

            # show the prediction on the frame
            if className!='rock':
                cv2.putText(image, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                     1, (200,20,198), 2, cv2.LINE_AA)


            # Show to screen

            cv2.imshow('gesture recognition', image)
            if(className=='peace'):
                webbrowser.open("youtube.com")
                break
            elif(className=='palm'):
                pyautogui.hotkey('alt', 'tab')
                pyautogui.hotkey('ctrl', 'tab')
                break
            elif(className=='rock'):
                webbrowser.open('google.com')
                break
            elif(className=='thumbs up'):
                pyautogui.hotkey('ctrl', 'shift','esc')
                break
            elif(className=='thumbs down'):
                pyautogui.hotkey('alt', 'tab')
                pyautogui.hotkey('alt', 'f4')
                break
                        


            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


startnew()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thumbs down
